# Knapsack

- Integrante 1: Alejandro Barrera
- Integrante 2: Steven Bernal
- Integrante 3: Camila Lenis
- Integrante 4: Javier Torres

The knapsack problem is a popular computer science problem. Part of the reason for it's popularity is it's combination of simple rules but difficult solution, as it has no solution in polynomial time. This problem is therefore an NP-complete problem, which means no correct solution exists that is also fast (polynomial time).

The game consists of the following:
- A knapsack (backpack) with a maximum capacity W
- N distinct items with a wegiht (Wi) and value (Vi)

Which of these items can you fit in the bag that maximises value (sum of items Vi) but does not exceed the maximum capacity W?

![img](https://upload.wikimedia.org/wikipedia/commons/f/fd/Knapsack.svg)

As an example, say you have items (Vi, Wi) and a knapsack with capacity of 15:
1. (4 \$, 12 kg)
2. (2 \$, 2 kg)
3. (2 \$, 1 kg)
4. (1 \$, 1 kg)
5. (10 \$, 4 kg)

Which combination of these achieves the best result? In this case, the answer is items 2-5, which add up to 15$ in value and 8kg in weight.

There is another variation of this problem, what if we could pick any amount of each item? In this case, we would add three of 5 and three of 3, which add up to 36$ with weight 15kg.

Your task is to write a genetic algorithm that computes the solution to the knapsack problem.

In [1]:
import random

class Item:
    def __init__(self, value, weight, quantity):
        self.weight = weight
        self.value = value
        self.quantity = quantity
    
    def __repr__(self):
        return 'Item: '+str(self.weight)+' - '+str(self.value)+' - '+str(self.quantity)

MAXIMUM_WEIGHT = 15
ITEMS = [Item(4, 12, 0), Item(2, 2, 0), Item(2, 1, 0), Item(1, 1, 0), Item(10, 4, 0)]

In [2]:
# takes a list items (one individual) and evaluates it: [Item(4, 12, 0), Item(2, 2, 0), Item(2, 1, 0), Item(1, 1, 0), Item(10, 4, 0)] -> X (depends on your fitness function)
def evaluate(items):
    total_value = 0
    total_weight = 0
    for it in items:
        total_value += it.quantity * it.value
        total_weight += it.quantity * it.weight
    if total_weight > MAXIMUM_WEIGHT:
        total_value = -total_weight
        
    return total_value # TODO: complete the fitness function

evaluate([Item(4, 12, 0), Item(2, 2, 1), Item(2, 1, 3), Item(1, 1, 0), Item(10, 4, 3)])

In [3]:
# takes a generation and evaluates them all, returning individuals in tuples with score: [
#  [Item(4, 12, 0), Item(2, 2, 0)],
#  [Item(4, 12, 0), Item(2, 2, 0)],
#  [Item(4, 12, 0), Item(2, 2, 0)]
#]
# ->
#[
#  ([Item(4, 12, 0), Item(2, 2, 0)], X),
#  ([Item(4, 12, 1), Item(2, 2, 0)], X),
#  ([Item(4, 12, 0), Item(2, 2, 1)], X)
#]
def evaluate_many(generation):
  return [ (individual, evaluate(individual)) for individual in generation ]

evaluate_many([[Item(4, 12, 0), Item(2, 2, 0)], [Item(4, 12, 0), Item(2, 2, 0)], [Item(4, 12, 0), Item(2, 2, 0)]])

In [4]:
# Takes a list of individuals and their scores and only returns the top_n ones [
#  ([Item(4, 12, 0), Item(2, 2, 0)], 1),
#  ([Item(4, 12, 1), Item(2, 2, 0)], 3),
#  ([Item(4, 12, 0), Item(2, 2, 1)], 2)
#]
# ->
#[
# ([Item(4, 12, 1), Item(2, 2, 0)], 3),
# ([Item(4, 12, 0), Item(2, 2, 1)], 2)
#]
def selection(evaluated_generation, top_n):
  evaluated_generation.sort(key = lambda e: e[1], reverse = True)

  return evaluated_generation[:top_n]

selection([
  ([Item(4, 12, 0), Item(2, 2, 0)], 1),
  ([Item(4, 12, 1), Item(2, 2, 0)], 3),
  ([Item(4, 12, 0), Item(2, 2, 1)], 2)
], 2)

In [5]:
# Takes an individual and mutates it n_genes times: [Item(4, 12, 0), Item(2, 2, 0)] -> [Item(4, 12, 3), Item(2, 2, 1)]
def mutate(individual, n_genes):
    new_individual = []
    for it in individual:
        delta = random.randint(-n_genes,n_genes)
        new_individual.append(Item(it.value,it.weight,max(0,it.quantity+delta)))
    return new_individual

mutate([Item(4, 12, 0), Item(2, 2, 0)], 2)

[Item: 12 - 4 - 0, Item: 2 - 2 - 2]

In [6]:
# Takes a list of individuals and generates a generation_size list of new mutated individuals 
def mutate_many(best_individuals, generation_size, n_genes):
  new_generation = []

  for i in range(generation_size):
    new_individual = mutate(random.choice(best_individuals), n_genes)
    new_generation.append(new_individual)
  
  #new_generation = new_generation + best_individuals
  #new_generation = selection(new_generation, generation_size)

  return new_generation

mutate_many([[Item(4, 12, 0), Item(2, 2, 0)], [Item(4, 12, 0), Item(2, 2, 0)]], 10, 1)

[[Item: 12 - 4 - 1, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 1],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 0, Item: 2 - 2 - 0],
 [Item: 12 - 4 - 0, Item: 2 - 2 - 1],
 [Item: 12 - 4 - 1, Item: 2 - 2 - 1]]

In [17]:
MAX_GENERATIONS = 50
TOP_N = 20
MUTATED_CHROMOSOMES = 2
GENERATION_SIZE = 20
FITNESS_THRESHOLD = 100


n_generation = 0
converged = False

generation = [ [Item(4, 12, 0), Item(2, 2, 0), Item(2, 1, 0), Item(1, 1, 0), Item(10, 4, 0)] for _ in range(GENERATION_SIZE)] # TODO: create initial generation

while n_generation < MAX_GENERATIONS and not converged:
  evaluated_generation = evaluate_many(generation)
  evaluated_best = selection(evaluated_generation, TOP_N)
  
  best_individuals = [ individual for individual, evaluation in evaluated_best ]

  generation = best_individuals + mutate_many(best_individuals, GENERATION_SIZE, MUTATED_CHROMOSOMES)
  
  best_in_generation = evaluated_best[0]
  converged = best_in_generation[1] >= FITNESS_THRESHOLD
  n_generation += 1

  print(f"{n_generation}: {best_in_generation}")

1: ([Item: 12 - 4 - 0, Item: 2 - 2 - 0, Item: 1 - 2 - 0, Item: 1 - 1 - 0, Item: 4 - 10 - 0], 0)
2: ([Item: 12 - 4 - 0, Item: 2 - 2 - 2, Item: 1 - 2 - 2, Item: 1 - 1 - 0, Item: 4 - 10 - 2], 28)
3: ([Item: 12 - 4 - 0, Item: 2 - 2 - 2, Item: 1 - 2 - 2, Item: 1 - 1 - 0, Item: 4 - 10 - 2], 28)
4: ([Item: 12 - 4 - 0, Item: 2 - 2 - 0, Item: 1 - 2 - 0, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 30)
5: ([Item: 12 - 4 - 0, Item: 2 - 2 - 1, Item: 1 - 2 - 0, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 32)
6: ([Item: 12 - 4 - 0, Item: 2 - 2 - 1, Item: 1 - 2 - 0, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 32)
7: ([Item: 12 - 4 - 0, Item: 2 - 2 - 1, Item: 1 - 2 - 0, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 32)
8: ([Item: 12 - 4 - 0, Item: 2 - 2 - 1, Item: 1 - 2 - 0, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 32)
9: ([Item: 12 - 4 - 0, Item: 2 - 2 - 0, Item: 1 - 2 - 2, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 34)
10: ([Item: 12 - 4 - 0, Item: 2 - 2 - 0, Item: 1 - 2 - 2, Item: 1 - 1 - 0, Item: 4 - 10 - 3], 34)
11: ([Item: 12 - 4 - 0, Item: 